In [2]:
import pandas as pd
import numpy as np
# Okay Looking at my data --> I have genres, tags, popularity and year proximity. I can use all of these to create a recommendation system.

In [3]:
df = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/movies.csv')
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [5]:
df2 = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/tags.csv')
df2.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
df3 = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/ratings.csv')
df3.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
## Steps:
## movies.csv: Create a year column and then normalize the year column - done 
## tags.csv: Concat the multiple tabs into one column and then pull it into movies.csv 
## ratings.csv: Group the ratings by movie --> Average + Count of ratings.  

In [8]:
df["year"] = df['title'].str.extract(r"\((\d{4})\)").astype(float)

In [10]:
df2

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [11]:
def group_tags(df, id_col="movieId", tag_col="tag"):
    """
    Groups a dataframe by `id_col` and concatenates the values
    in `tag_col` into a single string separated by '|'.
    """
    grouped = (
        df.groupby(id_col)[tag_col]
          .apply(lambda x: "|".join(x.astype(str)))
          .reset_index()
    )
    return grouped


In [ ]:
tags_grouped = group_tags(df2)
print(tags_grouped)

      movieId                                              tag
0           1                                  pixar|pixar|fun
1           2     fantasy|magic board game|Robin Williams|game
2           3                                        moldy|old
3           5                                 pregnancy|remake
4           7                                           remake
...       ...                                              ...
1567   183611                      Comedy|funny|Rachel McAdams
1568   184471  adventure|Alicia Vikander|video game adaptation
1569   187593                Josh Brolin|Ryan Reynolds|sarcasm
1570   187595                          Emilia Clarke|star wars
1571   193565                    anime|comedy|gintama|remaster

[1572 rows x 2 columns]


In [16]:
df_semiprepped = df.merge(tags_grouped, on="movieId", how="left")

In [17]:
df_semiprepped.head()

,movieId,title,genres,year,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,pixar|pixar|fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,fantasy|magic board game|Robin Williams|game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,moldy|old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,NaN
4,5,Father of the Bride Part II (1995),Comedy,1995.0,pregnancy|remake


In [18]:
df3.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [24]:
def group_ratings(df, id_col="movieId", rating_col="rating"):
    """
    Groups dataframe by movieId and computes:
      - average rating
      - count of ratings
    Returns a new dataframe with movieId, avg_rating, and rating_count.
    """
    grouped = (
        df.groupby(id_col)[rating_col]
          .agg(avg_rating="mean", rating_count="count")
          .reset_index()
    )
    grouped["avg_rating"] = grouped["avg_rating"].round(2)
    return grouped

In [25]:
ratings_grouped = group_ratings(df3)
ratings_grouped.head()

,movieId,avg_rating,rating_count
0,1,3.92,215
1,2,3.43,110
2,3,3.26,52
3,4,2.36,7
4,5,3.07,49


In [26]:
df_semiprepped_w_ratings = df_semiprepped.merge(ratings_grouped, on="movieId", how="left")

In [28]:
df_semiprepped_w_ratings

,movieId,title,genres,year,tag,avg_rating,rating_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,pixar|pixar|fun,3.92,215.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,fantasy|magic board game|Robin Williams|game,3.43,110.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,moldy|old,3.26,52.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,NaN,2.36,7.0
4,5,Father of the Bride Part II (1995),Comedy,1995.0,pregnancy|remake,3.07,49.0
...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017.0,NaN,4.00,1.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017.0,NaN,3.50,1.0
9739,193585,Flint (2017),Drama,2017.0,NaN,3.50,1.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018.0,NaN,3.50,1.0


In [29]:
## Okay so now I have a dataframe with (1) Title (2) Genres (3) Year (4) Tags (5) Average Rating (6) Number of Ratings
## Now i can start encoding the data.

In [44]:
df_semiprepped_w_ratings['year'] = df_semiprepped_w_ratings['year'].astype("Int64")
min_year, max_year = df_semiprepped_w_ratings["year"].min(), df_semiprepped_w_ratings["year"].max().astype(int)
df_semiprepped_w_ratings["year_scaled"] = (df_semiprepped_w_ratings["year"] - min_year) / (max_year - min_year)

In [45]:
df_semiprepped_w_ratings["year_scaled"].describe()

count      9729.0
mean     0.798393
std      0.159786
min           0.0
25%      0.741379
50%      0.836207
75%      0.913793
max           1.0
Name: year_scaled, dtype: Float64

In [ ]:
df_semiprepped_w_ratings['year'].nun()

2002    311
2006    295
2001    294
2007    284
2000    283
       ... 
1917      1
1915      1
1919      1
1921      1
1908      1
Name: year, Length: 106, dtype: Int64